<a href="https://colab.research.google.com/github/nikhilRajput-prog/Deep-Learning-Lab-File/blob/main/deep_learning_experiment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Experiment->4


#### Image classification using Convolutional Neural Networks (CNNs) to classify images. (Datasets: (Cats vs. Dogs) & (CIFAR-10)

In [ ]:
# Importing Libraries
from google.colab import drive
import os
import csv
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms, models

In [ ]:
print("Mounting Google Drive...")
drive.mount('/content/drive')
print("Google Drive mounted successfully!")

Mounting Google Drive...
Mounted at /content/drive
Google Drive mounted successfully!


In [ ]:
!ls '/content/drive/MyDrive'

 audio_files.zip        HateMM_annotation.csv   sampleSubmission.csv
'Colab Notebooks'       hate_videos.zip         test1.zip
'Getting started.pdf'   non_hate_videos.zip     train.zip


In [ ]:
!unzip -q /content/drive/MyDrive/test1.zip -d test-dataset

In [ ]:
!unzip -q /content/drive/MyDrive/train.zip -d train-dataset

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
print("Using device:", device)

Using device: cuda


In [ ]:
EPOCHS = 5
LR = 0.001
BATCH_SIZE = 64

In [ ]:
# Loading CIFAR10 dataset
def load_cifar10():
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5),
                             (0.5, 0.5, 0.5))
    ])

    train_set = datasets.CIFAR10(
        root="./data", train=True, download=True, transform=transform
    )
    test_set = datasets.CIFAR10(
        root="./data", train=False, download=True, transform=transform
    )

    train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

    return train_loader, test_loader

In [ ]:
#Importing labels from csv file for comparing test data of cat dog classification
def load_labels_from_csv(csv_path):
    label_dict = {}
    with open(csv_path, "r") as f:
        reader = csv.DictReader(f)
        for row in reader:
            label_dict[row["id"]] = int(row["label"])
    return label_dict

In [ ]:
class CatsDogsDataset(Dataset):
    def __init__(self, image_dir, transform=None, labels=None):
        self.image_dir = image_dir
        self.transform = transform
        self.images = os.listdir(image_dir)
        self.labels = labels
    def __len__(self):
        return len(self.images)
    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.image_dir, img_name)
        image = Image.open(img_path).convert("RGB")
        if self.labels is None:
            label = 0 if img_name.startswith("cat") else 1
        else:
            img_id = os.path.splitext(img_name)[0]
            label = self.labels[img_id]

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
# Load dataset
def load_cats_dogs():
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5),
                             (0.5, 0.5, 0.5))
    ])

    train_dataset = CatsDogsDataset(
        "/content/train-dataset/train",
        transform=transform
    )

    labels = load_labels_from_csv(
        "/content/drive/MyDrive/sampleSubmission.csv"
    )

    test_dataset = CatsDogsDataset(
        "/content/test-dataset/test1",
        transform=transform,
        labels=labels
    )

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    return train_loader, test_loader

In [ ]:
# CNN Model
class CustomCNN(nn.Module):
    def __init__(self, num_classes, activation, dataset_name):
        super(CustomCNN, self).__init__()
        if dataset_name == "cifar10":
            self.model = nn.Sequential(
                nn.Conv2d(3, 32, kernel_size=3, padding=1),
                nn.BatchNorm2d(32),
                activation,
                nn.MaxPool2d(2),

                nn.Conv2d(32, 64, kernel_size=3, padding=1),
                nn.BatchNorm2d(64),
                activation,
                nn.MaxPool2d(2),

                nn.Conv2d(64, 128, kernel_size=3, padding=1),
                nn.BatchNorm2d(128),
                activation,
                nn.MaxPool2d(2),

                nn.Flatten(),
                nn.Linear(128 * 4 * 4, 256),
                activation,
                nn.Dropout(0.5),
                nn.Linear(256, num_classes)
            )

        else:
            # Cats vs Dogs → 224x224 images
            self.model = nn.Sequential(
                nn.Conv2d(3, 32, kernel_size=3, padding=1),
                nn.BatchNorm2d(32),
                activation,
                nn.MaxPool2d(2),

                nn.Conv2d(32, 64, kernel_size=3, padding=1),
                nn.BatchNorm2d(64),
                activation,
                nn.MaxPool2d(2),

                nn.Conv2d(64, 128, kernel_size=3, padding=1),
                nn.BatchNorm2d(128),
                activation,
                nn.MaxPool2d(2),

                nn.Flatten(),
                nn.Linear(128 * 28 * 28, 256),
                activation,
                nn.Dropout(0.5),
                nn.Linear(256, num_classes)
            )

    def forward(self, x):
        return self.model(x)

In [ ]:
# Initializing weights
def init_weights(model, init_type):
    for layer in model.modules():
        if isinstance(layer, (nn.Conv2d, nn.Linear)):
            if init_type == "xavier":
                nn.init.xavier_uniform_(layer.weight)
            elif init_type == "kaiming":
                nn.init.kaiming_uniform_(layer.weight, nonlinearity="relu")
            else:
                nn.init.normal_(layer.weight, mean=0.0, std=0.02)
            if layer.bias is not None:
                nn.init.constant_(layer.bias, 0)

In [ ]:
def train_epoch(model, loader, optimizer, criterion):
    model.train()
    correct, total = 0, 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    return 100 * correct / total

In [ ]:
def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return 100 * correct / total

In [ ]:
def run(train_loader, test_loader, dataset_name, num_classes, save_path,EPOCHS):
    activations = {
        "ReLU": nn.ReLU(),
        "Tanh": nn.Tanh(),
        "LeakyReLU": nn.LeakyReLU(0.1)
    }
    weight_inits = ["xavier", "kaiming", "random"]
    optimizers = {
        "SGD": optim.SGD,
        "Adam": optim.Adam,
        "RMSprop": optim.RMSprop
    }
    criterion = nn.CrossEntropyLoss()
    best_test_acc = 0.0
    for act_name, act_fn in activations.items():
        for init_name in weight_inits:
            for opt_name, opt_class in optimizers.items():
                print(f"\nRunning: {act_name} | {init_name} | {opt_name}")
                model = CustomCNN(
                    num_classes=num_classes,
                    activation=act_fn,
                    dataset_name=dataset_name
                ).to(device)
                init_weights(model, init_name)
                optimizer = opt_class(model.parameters(), lr=LR)

                for epoch in range(EPOCHS):
                    train_acc = train_epoch(model, train_loader, optimizer, criterion)
                    test_acc = evaluate(model, test_loader)
                    print(
                        f"Epoch [{epoch+1}/{EPOCHS}] "
                        f"Train Acc: {train_acc:.2f}% | "
                        f"Test Acc: {test_acc:.2f}%"
                    )
                if test_acc > best_test_acc:
                    best_test_acc = test_acc
                    torch.save(model.state_dict(), save_path)
                    print("Best model saved")
    print(f"\nBest Test Accuracy: {best_test_acc:.2f}%")

In [ ]:
def train_resnet18(train_loader, test_loader, num_classes):
    model = models.resnet18(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    model = model.to(device)
    optimizer = optim.Adam(model.fc.parameters(), lr=LR)
    criterion = nn.CrossEntropyLoss()
    print("\nTraining ResNet-18")
    for epoch in range(EPOCHS):
        train_acc = train_epoch(model, train_loader, optimizer, criterion)
        test_acc = evaluate(model, test_loader)
        print(f"Epoch {epoch+1}: Train={train_acc:.2f}% Test={test_acc:.2f}%")

# Loading CIFAR dataset and running model

In [ ]:
print("                                         CIFAR-10")
cifar_train, cifar_test = load_cifar10()

                                         CIFAR-10


100%|██████████| 170M/170M [00:05<00:00, 29.5MB/s]


In [ ]:
images, labels = next(iter(cifar_train))
print(f"Shape of one image batch from cifar_train: {images.shape}")
print(f"Shape of one label batch from cifar_train: {labels.shape}")

images, labels = next(iter(cifar_test))
print(f"Shape of one image batch from cifar_test: {images.shape}")
print(f"Shape of one label batch from cifar_test: {labels.shape}")

Shape of one image batch from cifar_train: torch.Size([64, 3, 32, 32])
Shape of one label batch from cifar_train: torch.Size([64])
Shape of one image batch from cifar_test: torch.Size([64, 3, 32, 32])
Shape of one label batch from cifar_test: torch.Size([64])


In [ ]:
run(
    train_loader=cifar_train,
    test_loader=cifar_test,
    dataset_name="cifar10",
    num_classes=10,
    save_path="best_cnn_cifar10.pth",
    EPOCHS=5
)
train_resnet18(cifar_train, cifar_test, 10)


Running: ReLU | xavier | SGD
Epoch [1/5] Train Acc: 28.21% | Test Acc: 42.76%
Epoch [2/5] Train Acc: 38.59% | Test Acc: 47.86%
Epoch [3/5] Train Acc: 43.29% | Test Acc: 50.85%
Epoch [4/5] Train Acc: 46.05% | Test Acc: 52.77%
Epoch [5/5] Train Acc: 48.17% | Test Acc: 53.84%
Best model saved

Running: ReLU | xavier | Adam
Epoch [1/5] Train Acc: 41.30% | Test Acc: 59.34%
Epoch [2/5] Train Acc: 55.01% | Test Acc: 66.21%
Epoch [3/5] Train Acc: 60.74% | Test Acc: 68.10%
Epoch [4/5] Train Acc: 64.55% | Test Acc: 71.35%
Epoch [5/5] Train Acc: 67.03% | Test Acc: 71.74%
Best model saved

Running: ReLU | xavier | RMSprop
Epoch [1/5] Train Acc: 39.41% | Test Acc: 51.13%
Epoch [2/5] Train Acc: 55.66% | Test Acc: 62.23%
Epoch [3/5] Train Acc: 62.20% | Test Acc: 58.01%
Epoch [4/5] Train Acc: 66.46% | Test Acc: 70.32%
Epoch [5/5] Train Acc: 69.10% | Test Acc: 70.78%

Running: ReLU | kaiming | SGD
Epoch [1/5] Train Acc: 24.64% | Test Acc: 37.12%
Epoch [2/5] Train Acc: 33.31% | Test Acc: 41.71%
Epoch [

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [5/5] Train Acc: 75.85% | Test Acc: 67.15%

Best Test Accuracy: 77.41%
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 179MB/s]



Training ResNet-18
Epoch 1: Train=39.25% Test=45.23%
Epoch 2: Train=43.98% Test=46.49%
Epoch 3: Train=44.88% Test=45.52%
Epoch 4: Train=45.33% Test=45.69%
Epoch 5: Train=45.73% Test=46.38%


# Loading cat dog dataset and running model

In [ ]:
cd_train, cd_test = load_cats_dogs()

In [ ]:
run(
    train_loader=cd_train,
    test_loader=cd_test,
    dataset_name="catsdogs",
    num_classes=2,
    save_path="best_cnn_cats_dogs.pth",
    EPOCHS=3
)

                           Cats vs Dogs

Running: ReLU | xavier | SGD
Epoch [1/3] Train Acc: 64.43% | Test Acc: 59.40%
Epoch [2/3] Train Acc: 71.38% | Test Acc: 36.40%
Epoch [3/3] Train Acc: 74.67% | Test Acc: 59.18%
Best model saved

Running: ReLU | xavier | Adam
Epoch [1/3] Train Acc: 60.70% | Test Acc: 70.54%
Epoch [2/3] Train Acc: 62.27% | Test Acc: 70.07%
Epoch [3/3] Train Acc: 62.40% | Test Acc: 72.16%
Best model saved

Running: ReLU | xavier | RMSprop
Epoch [1/3] Train Acc: 56.39% | Test Acc: 45.31%
Epoch [2/3] Train Acc: 56.27% | Test Acc: 86.96%
Epoch [3/3] Train Acc: 58.29% | Test Acc: 68.40%

Running: ReLU | kaiming | SGD
Epoch [1/3] Train Acc: 63.98% | Test Acc: 73.65%
Epoch [2/3] Train Acc: 69.67% | Test Acc: 48.15%
Epoch [3/3] Train Acc: 73.03% | Test Acc: 50.66%

Running: ReLU | kaiming | Adam
Epoch [1/3] Train Acc: 58.95% | Test Acc: 66.30%
Epoch [2/3] Train Acc: 62.66% | Test Acc: 52.36%
Epoch [3/3] Train Acc: 64.47% | Test Acc: 63.02%

Running: ReLU | kaiming | RMSpro

In [ ]:
train_resnet18(cd_train, cd_test, 2)


Training ResNet-18
Epoch 1: Train=95.62% Test=50.42%
Epoch 2: Train=96.81% Test=50.31%
Epoch 3: Train=96.65% Test=51.65%
Epoch 4: Train=97.04% Test=50.17%
Epoch 5: Train=97.18% Test=49.94%
